# Session7-Assignment

In [82]:
import torch
import torchtext 
from torchtext import data, datasets
import pandas as pd
import nltk


In [83]:
df_dict = pd.read_csv("dictionary.txt", sep='\|', header=None)
df_label = pd.read_csv("sentiment_labels.txt", sep='|')
df_sentence = pd.read_csv("datasetSentences.txt", sep='\t')
df_split = pd.read_csv("datasetSplit.txt",sep=',')
df_org =pd.read_csv("original_rt_snippets.txt", sep='\t')

#df_original = pd.read_csv("original_rt_snippets.txt")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [84]:
print(f"Dictionary: {df_dict.shape}")
print(f"Sentimenet: {df_label.shape}")
print(f"Sentence: {df_sentence.shape}")
print(f"DataSplit: {df_split.shape}")
print(f"Org_Snippet: {df_org.shape}")

Dictionary: (239232, 2)
Sentimenet: (239232, 2)
Sentence: (11855, 2)
DataSplit: (11855, 2)
Org_Snippet: (10604, 1)


In [85]:
len(df_dict), len(df_label),len(df_sentence), len(df_split)



(239232, 239232, 11855, 11855)

In [86]:
df_dict.columns = ['phrases', 'phrase_id']

In [87]:
# Merging DataSentence & DataSplit with common column sentence_index
# df_sentence.columns, df_split.columns
DS01_merge = df_sentence.merge(df_split, on='sentence_index')

# Merging Dictionary with DataSentence & DataSplit 
DS02_merge = DS01_merge.merge(df_dict, left_on='sentence', right_on ='phrases', how='left')

# Merging label with DS_02_merge having dictionary, datasentence, datasplit
#df = pd.merge(df_phrase , df_label, how='inner', left_on=1, right_on='phrase ids' )
DS03_merge = DS02_merge.merge(df_label, left_on='phrase_id', right_on='phrase ids', how='left')
DS03_merge.head(5)



,sentence_index,sentence,splitset_label,phrases,phrase_id,phrase ids,sentiment values
0,1,The Rock is destined to be the 21st Century 's...,1,The Rock is destined to be the 21st Century 's...,226166.0,226166.0,0.69444
1,2,The gorgeously elaborate continuation of `` Th...,1,The gorgeously elaborate continuation of `` Th...,226300.0,226300.0,0.83333
2,3,Effective but too-tepid biopic,2,Effective but too-tepid biopic,13995.0,13995.0,0.51389
3,4,If you sometimes like to go to the movies to h...,2,If you sometimes like to go to the movies to h...,14123.0,14123.0,0.73611
4,5,"Emerges as something rare , an issue movie tha...",2,"Emerges as something rare , an issue movie tha...",13999.0,13999.0,0.86111


In [88]:
print(f"DS03_merge: {DS03_merge.shape}")
print(f"DS03_merge: {DS03_merge.columns}")

DS03_merge: (11855, 7)
DS03_merge: Index(['sentence_index', 'sentence', 'splitset_label', 'phrases', 'phrase_id',
       'phrase ids', 'sentiment values'],
      dtype='object')


In [89]:
DS03_merge['labels'] = [(1 if 0 <=i <=0.2 else (2 if 0.2<i<=0.4 else (3 if 0.4 <i<=0.6 else (4 if 0.6 <i<=0.8 else 5)))) for i in DS03_merge['sentiment values'] ]
DS03_merge.rename(columns={'sentence':'tweets'}, inplace=True)


In [90]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [91]:
# DATA Augmentation approaches
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
import random
from nltk.corpus import wordnet

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stoplist  = set(stopwords.words('english'))

stop = stopwords.words('english')
import string
punct = list(string.punctuation)
punct.remove("-")
punct.append(" ")


def get_synonyms(word):
  if word.lower() in stop:
      return [word], [1]

  synonyms = set()
  for syn in wordnet.synsets(word): 
      for l in syn.lemmas(): 
          synonym = l.name().replace("_", " ").replace("-", " ").lower()
          synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
          synonyms.add(synonym) 

  if word not in synonyms:
      synonyms.add(word)
      
  n = len(synonyms)

  if n == 1:
      word_ = "".join(list(filter(lambda x: x not in punct, word)))
      if word_.lower() in stop:
          return [word, word_], [0.5, 0.5]
      for syn in wordnet.synsets(word_): 
          for l in syn.lemmas(): 
              synonym = l.name().replace("_", " ").replace("-", " ").lower()
              synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
              synonyms.add(synonym) 
      if word_ not in synonyms:
          synonyms.add(word_)
          
  n = len(synonyms)
  if n == 1:
      probabilities = [1]
  else:
      probabilities = [0.5 if w==word else 0.5/(n-1) for w in synonyms]

  return list(synonyms), probabilities

def new_row(row, n_samples=1): 
  text = row['text']
  new_rows = [row]
  for i in range(n_samples):
      row2 = row
      new_words = []
      for word in text.split():
          syns, prob = get_synonyms(word)
          selected_word = np.random.choice(syns, p=prob, replace=True)
          if type(row['keyword']) == str:
              if word != selected_word and word in row['keyword']:
                  row2['keyword'] = row2['keyword'].replace(word, selected_word)
          new_words.append(selected_word)
          
      
      new_text = " ".join(new_words)
      
      row2['text'] = new_text
      new_rows.append(row2)

  new_rows = pd.concat(new_rows, axis=1).transpose().drop_duplicates(subset=['text'], inplace=False, ignore_index=True)
  new_rows = new_rows.loc[new_rows['text'].apply(len)<150]


  return new_rows


def remove_stopwords(text):
  return [word for word in word_tokenize(text) if not word in stoplist]

def random_swap(sentence, n=5): 
  length = range(len(sentence)) 
  for _ in range(n):
      idx1, idx2 = random.sample(length, 2)
      sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
  return sentence

def random_insertion(sentence, n): 
  words = remove_stopwords(sentence) 
  for _ in range(n):
      new_synonym = get_synonyms(random.choice(words))
      sentence.insert(randrange(len(sentence)+1), new_synonym) 
  return sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [92]:
x, y = get_synonyms('happy')
x, y

(['well chosen', 'felicitous', 'happy', 'glad'],
 [0.16666666666666666, 0.16666666666666666, 0.5, 0.16666666666666666])

In [93]:
final_df = DS03_merge[['tweets','labels', 'splitset_label']].copy()


In [94]:
final_df.shape

(11855, 3)

In [95]:
train_df, test_df, dev_df = [groups for name, groups in final_df.groupby('splitset_label')]

In [96]:
def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

In [97]:
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
dev_df.reset_index(drop=True, inplace=True)

train_df['tweets'] = train_df['tweets'].apply(lambda x:clean_text(x))

In [98]:
train_df.head()

,tweets,labels,splitset_label
0,The Rock is destined to be the 21st Century s...,4,1
1,The gorgeously elaborate continuation of The ...,5,1
2,Singer composer Bryan Adams contributes a slew...,4,1
3,You d think by now America would have had eno...,3,1
4,Yet the act is still charming here,4,1


In [99]:
# experimenting with one, random_insertion
ran_swapped_tweets = random_swap(train_df['tweets'], 5)

new_df = pd.DataFrame()
new_df['tweets'] = ran_swapped_tweets
new_df['labels'] = train_df['labels']
#new_df taken from below
new_df['splitset_label'] = train_df['splitset_label']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [100]:
train_df.shape, new_df.shape

((8544, 3), (8544, 3))

In [101]:
augmented_train_df = pd.concat([train_df[['tweets','labels','splitset_label']], new_df], ignore_index=True)
print("train_data count", augmented_train_df.labels.value_counts(), augmented_train_df.shape)

train_data count 4    4432
2    4184
5    3314
3    3098
1    2060
Name: labels, dtype: int64 (17088, 3)


In [102]:
!pip install nlpaug
!pip install fairseq
!pip install sacremoses

In [103]:
import nlpaug.augmenter.word as naw
#import fairseq
#from fairseq.models.transformer import TransformerModel

#aug = naw.BackTranslationAug()
#aug = naw.SynonymAug()

In [104]:


augmented_synonyms = [aug.augment(i,num_thread=32) for i in train_df['tweets']]
augment_synonyms_df = pd.DataFrame()
augment_synonyms_df['tweets'] = augmented_synonyms
augment_synonyms_df['labels'] = train_df['labels']
#new_df taken from below
augment_synonyms_df['splitset_label'] = train_df['splitset_label']

In [105]:
augmented_train_df = pd.concat([augmented_train_df, augment_synonyms_df], ignore_index=True)

In [106]:
augmented_train_df.to_csv("augmented_data.csv")
print("Completed")

Completed


In [107]:
!pip install fastBPE

In [108]:
#aug = naw.BackTranslationAug()

In [109]:
augment_bt_df = pd.DataFrame()
augment_bt_df['tweets'] = train_df['tweets'].apply(lambda x:aug.augment(x))
augment_bt_df['labels'] = train_df['labels']
#new_df taken from below
augment_bt_df['splitset_label'] = train_df['splitset_label']

In [110]:
augmented_train_df = pd.concat([augmented_train_df, augment_bt_df], ignore_index=True)

In [111]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [112]:
fields = [('tweets', Tweet),('labels',Label)]
final_df.head()
df = final_df[['tweets','labels']].copy()
train_df = augmented_train_df.copy()

In [113]:
train_example = [data.Example.fromlist([train_df.tweets[i],train_df.labels[i]], fields) for i in range(train_df.shape[0])] 
test_example = [data.Example.fromlist([test_df.tweets[i],test_df.labels[i]], fields) for i in range(test_df.shape[0])] 
valid_example = [data.Example.fromlist([dev_df.tweets[i],dev_df.labels[i]], fields) for i in range(dev_df.shape[0])] 

In [114]:
train = data.Dataset(train_example, fields)
valid = data.Dataset(valid_example, fields)
test = data.Dataset(test_example, fields)

In [115]:
(len(train), len(valid), len(test))

(34176, 1101, 2210)

In [116]:
vars(train.examples[10])

{'labels': 5,
 'tweets': ['Good',
  'fun',
  ' ',
  'good',
  'action',
  ' ',
  'good',
  'acting',
  ' ',
  'good',
  'dialogue',
  ' ',
  'good',
  'pace',
  ' ',
  'good',
  'cinematography']}

In [117]:
#!ls drive/MyDrive/data/.vector_cache


glove.6B.100d.txt     glove.6B.200d.txt  glove.6B.50d.txt
glove.6B.100d.txt.pt  glove.6B.300d.txt  glove.6B.zip


In [118]:
#glove_path = "drive/MyDrive/data/.vector_cache"

In [119]:
MAX_VOCAB_SIZE = 25_000

Tweet.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

Label.build_vocab(train)

2020-12-16 21:04:43 | INFO | torchtext.vocab | Loading vectors from .vector_cache/glove.6B.100d.txt.pt


In [120]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  24645
Size of label vocab :  5
Top 10 words appreared repeatedly : [(' ', 25142), ('the', 24557), ('of', 18237), ('and', 18067), ('a', 18055), ('to', 12405), ('s', 9853), ('is', 8642), ('that', 7830), ('in', 7552)]
Labels :  defaultdict(<function _default_unk_index at 0x7fcae902b400>, {4: 0, 2: 1, 5: 2, 3: 3, 1: 4})


Lots Of Stopwords

In [121]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [122]:
ls -a

./                    datasetSplit.txt          sentiment_labels.txt
../                   dictionary.txt            SOStr.txt
augmented_data.csv    drive/                    STree.txt
.config/              original_rt_snippets.txt  tokenizer.pkl
datasetSentences.txt  sample_data/              .vector_cache/


In [123]:
cp -r .vector_cache drive/MyDrive/data/.vector_cache


In [124]:
batch_size=64

train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = batch_size, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

train_iterator, test_iterator = data.BucketIterator.splits((train, test), batch_size = batch_size, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)


In [125]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

Defining Our Model

In [126]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout, pad_idx, bidirectional=False):
        
        super().__init__()          
        
        self.bidirectional = bidirectional
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           bidirectional=bidirectional,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.dropout = nn.Dropout(dropout)
        scaling_factor = 2 if bidirectional else 1
        
        self.fc = nn.Linear(hidden_dim*scaling_factor, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.dropout(self.embedding(text))
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        if self.bidirectional:
          hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
          hidden = self.dropout(hidden)
          hidden = hidden[0]
          
        dense_outputs = self.fc(hidden)   
        # Final activation function softmax
        output = F.softmax(dense_outputs, dim=1)
            
        return output

In [127]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 100
num_hidden_nodes = 256
num_output_nodes = 5
num_layers = 2
dropout = 0.5
bidirectional=True
PAD_IDX = Tweet.vocab.stoi[Tweet.pad_token]


# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout, pad_idx=PAD_IDX, bidirectional=bidirectional)

In [128]:
PAD_IDX = Tweet.vocab.stoi[Tweet.pad_token]

UNK_IDX = Tweet.vocab.stoi[Tweet.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[PAD_IDX] = torch.zeros(embedding_dim)

In [129]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(24645, 100, padding_idx=1)
  (encoder): LSTM(100, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=5, bias=True)
)
The model has 4,777,209 trainable parameters


Model Training and Evaluation

In [130]:
import torch.optim as optim

# define optimizer and loss
# lr=0.005
# lr = 2e-4
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# criterion = nn.BCELoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    _, predictions = torch.max(preds, 1)
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

Train Loop

In [131]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        #acc = binary_accuracy(predictions, batch.labels)   
        acc = categorical_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Evaluation Loop

In [132]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            # acc = binary_accuracy(predictions, batch.labels)
            acc = categorical_accuracy(predictions, batch.labels)   
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Let's Train and Evaluate

In [133]:
N_EPOCHS = 20
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.546 | Train Acc: 32.11%
	 Val. Loss: 1.575 |  Val. Acc: 28.27% 

	Train Loss: 1.492 | Train Acc: 39.02%
	 Val. Loss: 1.546 |  Val. Acc: 33.29% 

	Train Loss: 1.444 | Train Acc: 44.58%
	 Val. Loss: 1.562 |  Val. Acc: 31.75% 

	Train Loss: 1.408 | Train Acc: 48.54%
	 Val. Loss: 1.548 |  Val. Acc: 33.57% 

	Train Loss: 1.373 | Train Acc: 52.23%
	 Val. Loss: 1.566 |  Val. Acc: 32.97% 

	Train Loss: 1.341 | Train Acc: 55.32%
	 Val. Loss: 1.543 |  Val. Acc: 34.97% 

	Train Loss: 1.315 | Train Acc: 58.20%
	 Val. Loss: 1.547 |  Val. Acc: 34.18% 

	Train Loss: 1.291 | Train Acc: 60.67%
	 Val. Loss: 1.558 |  Val. Acc: 34.00% 

	Train Loss: 1.275 | Train Acc: 62.22%
	 Val. Loss: 1.555 |  Val. Acc: 33.58% 

	Train Loss: 1.257 | Train Acc: 64.31%
	 Val. Loss: 1.543 |  Val. Acc: 34.97% 

	Train Loss: 1.238 | Train Acc: 66.12%
	 Val. Loss: 1.533 |  Val. Acc: 35.48% 

	Train Loss: 1.223 | Train Acc: 67.72%
	 Val. Loss: 1.543 |  Val. Acc: 35.06% 

	Train Loss: 1.213 | Train Acc: 68.71%
	

In [134]:
model.load_state_dict(torch.load('saved_weights.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.533 | Test Acc: 35.90%


In [135]:
path = './sentiment_analysis_saved_weights_1.pt'

In [136]:
#load weights and tokenizer

path = './sentiment_analysis_saved_weights_1.pt'
# path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    # labels: very negative, negative, neutral, positive, very positive
    categories = {1: "very negative", 2:"negative", 3:"neutral", 4:'positive', 5: 'very positive'}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

FileNotFoundError: ignored

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

Random Insertion

In [ ]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

Random Deletion

In [ ]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

Random Swap

In [ ]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

Back Translation

In [ ]:
import random
import googletrans
import googletrans.Translator

translator = Translator()
sentence = ['The dog slept on the rug']

available_langs = list(googletrans.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, dest=trans_lang) 
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
en_text = [t.text for t in translations_en_random]
print(en_text)